In [1]:
from pathlib import Path
import sys
path_root = Path().absolute().parent
sys.path.append(str(path_root))

import numpy as np
import pandas as pd
import pickle
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

from radiomicsutil.voi import find_contours, getMask, calc_vol, calc_mesh, smooth_mesh, plot_mesh, plot_mesh3d
from radiomicsutil.threed import ThreedMorphologyFeaturesExtractor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
with open("./data/p_slice_locations.pkl", "rb") as f:
    case_slice_locations = pickle.load(f)

with open("./data/p_pixel_spacings.pkl", "rb") as f:
    case_pixel_spacings = pickle.load(f)


def caseExtractFeatures(isd_number, image_shape=(512,512)):
    print(isd_number)
    case = Path(f"/home/azetry/datasets/TACE/{isd_number}_result")
    
    pixel_spacing_xy = case_pixel_spacings[isd_number][0]

    # Convert slice locations to float and sort along with masks
    slice_locations = case_slice_locations[isd_number]
    sorted_indices = np.argsort(slice_locations)
    slice_locations.sort()


    with open(case/"liver_preds.pickle", "rb") as f:
        liver_preds = pickle.load(f)

    tumor_preds = dict()
    for png in case.glob("./*.png"):
        idx = int(png.name.replace(".png",""))
        img = cv2.imread(str(png))
        tumor_preds[idx] = find_contours(img)

    liver_masks = [ getMask(ctr,image_shape) for ctr in liver_preds ]
    liver_masks = [liver_masks[i] for i in sorted_indices]

    tumor_masks = [ getMask(tumor_preds[idx],image_shape) for idx in range(len(tumor_preds)) ] # 因為排序不一定照排序
    tumor_masks = [tumor_masks[i] for i in sorted_indices]

    # Default rescale to 1 mm
    liver_vol = calc_vol(liver_masks, slice_locations, pixel_spacing_xy)
    tumor_vol = calc_vol(tumor_masks, slice_locations, pixel_spacing_xy)

    # print("Building Liver Model...")
    # liver_extractor = ThreedMorphologyFeaturesExtractor(liver_vol)
    # print("Done.")
    # print("Building Tumor Model...")
    tumor_extractor = ThreedMorphologyFeaturesExtractor(tumor_vol)

    threed_morphology_p = tumor_extractor.all('p')
    
    # 用 triangle mesh 計算volume所花成本和時間太大，直接用 voxels 計算近似值 (因為我們已經rescale 到 1 mm 所以不用擔心)
    liver_voxels = np.sum(liver_vol)
    tumor_voxels = np.sum(tumor_extractor.voi_voxels)
    print(liver_voxels, tumor_voxels)
    # print(f"Liver voxels:{liver_voxels}")
    threed_morphology_p['ｐ_tumor_liver_volume_ratio'] = pd.Series([tumor_voxels / liver_voxels])
    threed_morphology_p['p_tumor_liver_diff'] = pd.Series([liver_voxels-tumor_voxels])
    threed_morphology_p['p_tumor_liver_diff_ratio'] = threed_morphology_p['p_tumor_liver_diff'] / liver_voxels

    threed_morphology_p = threed_morphology_p.set_index(pd.Index([f'{isd_number}']))
    return threed_morphology_p
    

In [7]:
with open("../20230604-iclab-radiomics/data/001-max-contours.pickle", "rb") as f:
    contours = pickle.load(f)

cases = [ case[0] for case in contours ]
print(f"cases with ct: {len(cases)}")

cases with ct: 517


In [8]:
r = caseExtractFeatures('TACE0104')

TACE0104
Use voxel to estimate volume...
1686638 858448


In [9]:
r

,p_counts,p_volume,p_surface_area,p_equivalent_diameter,p_extent,p_principal_axis_length_1,p_principal_axis_length_2,p_principal_axis_length_3,p_quaternion_w,p_quaternion_x,...,p_aspect_ratio_3,p_max_radius,p_abb_volume,p_sphericity_ellipticity,p_defect_ratio,p_gauss_curv_mean,p_mean_curv_mean,ｐ_tumor_liver_volume_ratio,p_tumor_liver_diff,p_tumor_liver_diff_ratio
TACE0104,3,845222,48920.638786,117.307035,0.473361,42.844719,30.574794,29.980817,0.58091,-0.517403,...,0.980573,87.822584,1785576.0,0.058942,0.702105,0.009237,-0.026151,0.50897,828190,0.49103


In [5]:
results = list()

for idx, isd_number in enumerate(cases):
    results.append(caseExtractFeatures(isd_number))

    if (idx+1)%10 == 0:
        pd.concat(results, axis=0).to_csv("./data/007-threed-morphology.csv")

pd.concat(results, axis=0).to_csv("./data/007-threed-morphology.csv")

print("Done.")

TACE0001
TACE0002
TACE0005
